# Scraping LADOT Volume Data from PDFs, Part 2

##### Where I Left Off
In the first python notebook, I described the problem and general approach I was taking to solve it. I downloaded all the manual count PDF documents and setup the file tables for later.

## PDF Text Extraction Process
As I discussed in the first python notebook, my process for extracting the text into usable data can be broken down into the few key parts: (1) define bounding boxes, (2) search for text within the bounding boxes (3) reformat the resulting text into multiple data tables, and (4) join the resulting tables to the ID established by the Bureau of Engineering. 

##### Define Bounding Boxes
This was tricky. I initially began defining bounding boxes using pixel measurements from a few sample pages. However, I quickly realized that due to the second of the challenges I mentioned above that this would not work, since the tables are in different locations among the PDF documents. 

Instead, I decided to create bounding boxes on the fly for each document using relative positioning of certain keywords that appeared almost always on each PDF document. Using pdfquery, I could begin by searching the document for these keywords and then extract the x,y pixel coordinate locations for the bounding box of each one. By getting the coordinates of multiple keyword objects on the page, I could construct a set of bounding boxes that seemed to perform relatively well in capturing data tables.

(create image of what this looked like)

##### Search for Text within Bounding Boxes
Once I had the coordinates of the bounding boxes, this part was quite easy, using PDFQuery to extract text

(do i need to adjust any of the parameters in rapidminer??)

##### Reformat the Resulting Text into Data Tables
The final problem included taking the scraped text from the bounding boxes and reformatting them into usable data tables. I kept in mind the relational database model as I set the format for these tables. From the PDF image above, I decided on the following tables and attributes:

*tbl_manualcount_info:* This table contains the basic information about the manual count summary. Each count will have one tuple with the following information:
* street_ns: The North / South Street running through the intersection
* street_ew: The East / West street running through the intersection
* dayofweek: Day of the Week
* date: Date, in datetime format
* weather: Prevailing weather at the time of the count (Clear, Sunny, etc.)
* hours: the hours of the count (text)
* school_day: A Yes / No indication of whether the count occurred on a school day. This is important because it heavily affects the volume counts
* int_code: The "I/S Code" on the form corresponds to the CL_Node_ID on the BOE Centerline. This ID field makes it easy to join to the City's centerline network
* district: The DOT field district in which the count took place
* count_id: Unique identifier assigned to the summary

*tbl_manualcount_dualwheel:* This table contains count data for dual-wheeled (motorcycles), bikes, and buses. Each form will have 12 tuples with the following information:
* count_id: Unique identifier assigned to the summary in "tbl_manual_count_info"
* approach: Intersection approach being measured (N,S,E,W)
* type: Dual-Wheeled / Bikes / Buses
* volume: Count

*tbl_manualcount_peak:* This table contains the peak hour / 15 minute counts. Each form will have 16 tuples with the following information:
* count_id: Unique identifier assigned to the summary in "tbl_manual_count_info"
* approach: Intersection approach being measured (N,S,E,W)
* type: The type of count
    * am_15: The AM peak 15 minute count
    * am_hour: The AM peak hour count
    * pm_15: The PM peak 15 minute count
    * pm_hour: The PM peak hour count
* time: Time of each count (in datetime format)
* volume: Count

*tbl_manualcount_volumes:* This table contains the main volume counts for each approach at the intersection. The number of tuples for each form will vary depending on the number of hours surveyed. A 6-hour count will have 6 hours * 3 directions (left, right, through) * 4 approach directions = 54 tuples. Each tuple will have the following information:
* count_id: Unique identifier assigned to the summary in "tbl_manual_count_info"
* approach: Northbound (NB) / Southbound (SB) / Eastbound (EB) / Westbound (WB)
* movement: Right-Turn (Rt) / Through (Th) / Left-Turn (Lt)
* start_time: Start time of that count, in datetime format
* end_time: End time of that count, in datetime format
* volume: Count

*tbl_manualcount_peds:* This table contains pedestrian and schoolchildren counts during the same time as the main volume counts, so the number of tuples will also be dependent on the number of hours the location was surveyed. Each tuple will have the following information:
* count_id: Unique identifier assigned to the summary in "tbl_manual_count_info"
* xing_leg: The leg of the intersection that is being crossed. South Leg (SL) / North Leg (NL) / West Leg (WL) / East Leg (EL)
* type: Ped / Sch
* start_time: Start time of that count, in datetime format
* end_time: End time of that count, in datetime format
* volume: Count


In [1]:
### Setup
import csv
import glob
from datetime import datetime, date, time
import pdfquery
import pandas as pd
import numpy as np
import os
from pathlib2 import Path

### Load NavLA Count file table
count_files_df = pd.read_csv('data/TrafficCountFileStructure/navLAfiles.csv',index_col=0)

count_files_df.head()

,cl_node_id,location,traffic_id,type,file
count_id,,,,,
1,3667,ISLAND AVE at L ST,1,manual,3667_ISLLST94.PDF
2,3680,FIGUEROA ST at L ST,2,manual,3680_FIGLST94.PDF
3,3727,FRIES AVE at HARRY BRIDGES BLVD,3,manual,3727_FRIHAR95.PDF
4,3787,ANAHEIM ST AT FARRAGUT AVE,4,manual,3787_ANAFAR100817.PDF
5,3787,ANAHEIM ST AT FARRAGUT AVE,4,manual,FARRAGUT.ANAHEIM.110119-MAN.PDF


### Step 1: Prepare the Data Tables
Based on the outline above, the first step is to prepare dataframes for each of the volume-related tables discussed above. Once I have the dataframe table structures, the next step involves looping through all the manual count PDFs, running my text extract function, and then inserting the rows into the appropriate tables. 

In [2]:
import pandas as pd

### Load NavLA Count file table
count_files_df = pd.read_csv('data/TrafficCountFileStructure/navLAfiles.csv',index_col=0)
manual_count_files_df = count_files_df[(count_files_df['type'] == 'manual')].reset_index()

manual_count_files_df.head()

,count_id,cl_node_id,location,traffic_id,type,file
0,1,3667,ISLAND AVE at L ST,1,manual,3667_ISLLST94.PDF
1,2,3680,FIGUEROA ST at L ST,2,manual,3680_FIGLST94.PDF
2,3,3727,FRIES AVE at HARRY BRIDGES BLVD,3,manual,3727_FRIHAR95.PDF
3,4,3787,ANAHEIM ST AT FARRAGUT AVE,4,manual,3787_ANAFAR100817.PDF
4,5,3787,ANAHEIM ST AT FARRAGUT AVE,4,manual,FARRAGUT.ANAHEIM.110119-MAN.PDF


### Step 2: Run the PDF text scraping script (extract / transform)
The powerhouse behind this process is the script I built to read a PDF and extract text into a list of dictionaries (one for each table). Because this script is so long, I opted not to include the code in the notebook, and instead treated it as a module that I imported using "import VolumeCountSheets_V2" above. I call the "pdf_extract" function from the module on each PDF.

### Step 3: Append each resulting dictionary to the Pandas dataframe (load)
The function returns a list of dictionaries, one for each dataframe. I then take the pandas dataframes I constructed in the cell above and append each dictionary to the appropriate one.

In [4]:
# Import PDF Scrape Module
import VolumeCountSheets_V2
import VolumeCountSheets_V3

# Setup Counter of sucessful / failed attempts
success = 0
failures = 0

# Create empty lists for storing count data
manualcount_volumes = []
ped_volumes = []
peak_volumes = []
info = []
specveh_volumes = []
qa = []

#### Loop through files, Run function
for index, row in manual_count_files_df.iterrows():
    
    Manual_TC = []
    
    print(row['file'])
#     if index > 200:
#         break
    
    # Folder location of all the count PDFs
    fileloc = 'data/TrafficCountData/Manual/All/' + row['file']
    #fileloc = 'data/TrafficCountData/Manual/All/' + '4347_IMPMON97.pdf'
    
    # ID for the count
    count_id = row['count_id']
    if index%100 == 0:
        print("Current Count:" + str(index+1))

    if Path(fileloc).exists():

        # Run the extract function
        try:
            Manual_TC = VolumeCountSheets_V2.pdf_extract(fileloc)
        except:
            try:
                print('trying v3')
                Manual_TC = VolumeCountSheets_V3.pdf_extract(fileloc)
            except:
                pass
                #raise
                
        # Add data if successful
        if len(Manual_TC) > 0:
                       
            # Append each row to our lists
            #if row['file'] == '4347_IMPMON97.PDF':
            #print(Manual_TC)
            for m in Manual_TC['Spec_Veh']:
                m['count_id'] = str(count_id)
                specveh_volumes.append(m)
            
            for j in Manual_TC['Volume']:
                j['count_id'] = str(count_id)
                manualcount_volumes.append(j)
                
            for k in Manual_TC['Pedestrian']:
                k['count_id'] = str(count_id)
                ped_volumes.append(k)
                
            for l in Manual_TC['PeakVol']:
                l['count_id'] = str(count_id)
                peak_volumes.append(l)
            
            Manual_TC['QA']['count_id'] = str(count_id)
            qa.append(Manual_TC['QA'])
                
            Manual_TC['Info']['count_id'] = str(count_id)
            info.append(Manual_TC['Info'])
            
            # Update success counter
            success+=1
            print('success')
               
        else:
            failures += 1
    else:
        print('path does not exist')
            
# Create dataframes from lists of dictionaries
info_df = pd.DataFrame.from_records(info)
manualcount_df = pd.DataFrame.from_records(manualcount_volumes)
pedestrian_df = pd.DataFrame.from_records(ped_volumes)
peakvol_df = pd.DataFrame.from_records(peak_volumes)
specveh_df = pd.DataFrame.from_records(specveh_volumes)
qa_df = pd.DataFrame.from_records(qa)

# Save dataframes to CSVs
manualcount_df.to_csv(path_or_buf='data/TrafficCountData/Results/manualcount_test.csv',sep=',')
pedestrian_df.to_csv(path_or_buf='data/TrafficCountData/Results/pedestrian_test.csv',sep=',')
peakvol_df.to_csv(path_or_buf='data/TrafficCountData/Results/peakvol_test.csv',sep=',')
specveh_df.to_csv(path_or_buf='data/TrafficCountData/Results/SpecialVehicle_test.csv',sep=',')
info_df.to_csv(path_or_buf='data/TrafficCountData/Results/info_test.csv',sep=',')
qa_df.to_csv(path_or_buf='data/TrafficCountData/Results/qa_test.csv',sep=',')

# Print the dataframes here
print("Success Count")
print(str(success))
print("Failure Count")
print(failures)


3667_ISLLST94.PDF
Current Count:1
trying v3
success
3680_FIGLST94.PDF
trying v3
success
3727_FRIHAR95.PDF
trying v3
success
3787_ANAFAR100817.PDF
trying v3
FARRAGUT.ANAHEIM.110119-MAN.PDF
success
3839_DONFRI93.PDF
trying v3
success
3975_ORCRED060627.PDF
trying v3
3976_ORCRED060627.PDF
trying v3
3976_ORCRED99.PDF
trying v3
success
REDONDOBEACH.ORCHARD.150616-NDSMAN.PDF
success
3995_GARMEN04.PDF
trying v3
success
3995_GARMEN98.PDF
trying v3
success
GARDENA.MENLO.171019-MAN.PDF
success
4007_ESTGAR95.PDF
trying v3
success
ESTGAR06.PDF
trying v3
4009_HOO16893.PDF
trying v3
success
4009_HOO16894.PDF
trying v3
success
4069_IMPSTA04.PDF
trying v3
success
4089_IMPWAD04.PDF
trying v3
success
WADSWORTH.114TH ST_IMPERIAL HWY.171025-NDSMAN.PDF
success
4091_FIGIMP95.PDF
trying v3
success
4091_FIGIMP98.PDF
trying v3
success
4202_HOO82N93.PDF
trying v3
success
4203_HOO83R93.PDF
trying v3
success
4265_FIG84S94.PDF
trying v3
success
4279_­­HOO95S.PDF
trying v3
CENHOO080506.PDF
success
4335_GRA11493.PDF


trying v3
6222_INDPER01.PDF
trying v3
success
6222_INDPER04.PDF
trying v3
success
6243_CALOLY080513.PDF
trying v3
6249_SOTWAS00.PDF
trying v3
success
6249_SOTWAS03.PDF
trying v3
success
6249_SOTWAS080513.PDF
trying v3
6249_SOTWAS94.PDF
trying v3
success
6251_INDOLY080513.PDF
trying v3
6277_INDWHI080508.PDF
trying v3
INDWHI06.PDF
trying v3
6278_CALWHI080507.PDF
trying v3
6285_OLYSAN01.PDF
trying v3
success
6285_OLYSAN080507.PDF
trying v3
6285_OLYSAN94.PDF
trying v3
success
OLYMPIC.SANTAFE.150422-MAN.PDF
success
6286_PORSAN00.PDF
trying v3
success
6286_PORSAN080507.PDF
trying v3
6309_HOO53S01.PDF
trying v3
success
6309_HOO53S96.PDF
trying v3
success
6310_HOL53R93.PDF
trying v3
success
6310_HOL53S99.PDF
trying v3
success
7641_NORVEN110707.PDF
trying v3
7642_ARLVEN110707.PDF
trying v3
ARLINGTON.VENICE.110712-MAN.PDF
success
7665_AV17S01.PDF
trying v3
success
4TH.17TH.130926-MAN.PDF
success
7667_6AV17S95.PDF
trying v3
success
7667_6AV17S98.PDF
trying v3
success
7667_6TH17T92.PDF
trying v3
s

trying v3
8440_DIAFIG00.PDF
trying v3
success
DIAFIG080701.PDF
success
DIAMOND.FIGUEROA.150616-NDSMAN.PDF
success
8444_CESPRI110707.PDF
trying v3
8444_CESSPR99.PDF
trying v3
success
CESSPR090217.PDF
success
8448_ALVWI195.PDF
trying v3
success
8448_ALVWI295.PDF
trying v3
success
8448_ALVWI495.PDF
trying v3
success
8448_ALVWIL01.PDF
trying v3
success
8448_ALVWIL110407.PDF
trying v3
8448_ALVWIL95.PDF
trying v3
success
8448_ALVWIL96.PDF
trying v3
success
8448_ALVWIL97.PDF
trying v3
success
8448_ALVWIL98.PDF
trying v3
ALVARADO.WILSHIRE.150421-WILMAN.PDF
success
ALVARADO.WILSHIRE.150423-WILMAN.PDF
success
ALVARADO.WILSHIRE.110527-MAN.PDF
success
ALVARADO.WILSHIRE.110426-MAN.PDF
success
ALVARADO.WILSHIRE.130416-MAN-NDS.PDF
success
ALVARADO.WILSHIRE.070425-MAN.PDF
trying v3
ALVARADO.WILSHIRE.070426-MAN.PDF
trying v3
ALVARADO.WILSHIRE.130417-MAN-NDS.PDF
success
ALVARADO.WILSHIRE.090415-MAN.PDF
trying v3
ALVARADO.WILSHIRE.090416-MAN.PDF
trying v3
8451_1STDEW090423.PDF
trying v3
8452_3RDBIX101215

success
10192_GAF22S95.PDF
trying v3
success
10193_GRA22S04.PDF
trying v3
success
10204_GAF23S98.PDF
trying v3
success
10224_GAF25S95.PDF
trying v3
success
10224_GAF25S98.PDF
trying v3
success
10301_GAF30S95.PDF
trying v3
success
GAF30S07.PDF
trying v3
10346_GAFPAS93.PDF
trying v3
success
10491_ABBCAD04.PDF
trying v3
success
10497_ABBAND02.PDF
trying v3
success
10497_ABBAND04.PDF
trying v3
success
10782_DRIPAC04.PDF
trying v3
success
10824_ABBMAR05.PDF
trying v3
success
10824_ABBMIL02.PDF
trying v3
success
ABBOTKINNEY.MARR140125-MAN.PDF
trying v3
ABBOTKINNEY.MARR131204-MAN.PDF
success
10947_JEFMES94.PDF
trying v3
success
10947_JEFMES98.PDF
trying v3
success
10961_GLEMIN95.PDF
trying v3
success
10961_GLEMIN97.PDF
trying v3
success
10961_MINGLE110412.PDF
trying v3
10973_JEFMAR02.PDF
trying v3
success
JEFMAR080910.PDF
success
11140_GLEMAX110414.PDF
trying v3
11140_GLEMAX94.PDF
trying v3
success
11140_GLEMAX97.PDF
trying v3
success
11140_GLEMBM94.PDF
trying v3
success
CULVER.JEFFERSON.1707

trying v3
14268_BARLAG060406.PDF
trying v3
BARRINGTON.LAGRANGE.120517-MAN.PDF
trying v3
BARRINGTON.LAGRANGE.130529.NDS_MAN.PDF
success
BARRINGTON_LAGRANGE_2013.PDF
success
14281_CENIDA060315.PDF
trying v3
CENTINELA.COLORADO & IDAHO.120509.NDS_MAN.PDF
trying v3
CENTINELA_IDAHO_2012.PDF
trying v3
BARRINGTON_MISSISSIPPI_2013.PDF
success
BARRINGTON.MISSISSIPPI.110907-MAN.PDF
trying v3
14284_BARMIS060302.PDF
trying v3
14284_BARMIS95.PDF
trying v3
success
BARRINGTON.MISSISSIPPI.130529.NDS_MAN.PDF
success
14285_FEDOLY94.PDF
trying v3
success
BUNDY.MISSOURI.110908-MAN.PDF
trying v3
BUNDY.MISSOURI.120516-MAN.PDF
trying v3
14289_BUNMIS060314.PDF
trying v3
BUNDY.MISSOURI.120509.NDS_MAN.PDF
trying v3
BUNDY_MISSOURI_2012.PDF
trying v3
14290_BUNNEB060314.PDF
trying v3
BUNDY.NEBRASKASOUTH.120509.NDS_MAN.PDF
trying v3
BUNDY_NEBRASKA_SOUTH_2012.PDF
trying v3
BUNDY.NEBRASKA.120619-MAN.PDF
success
BARRINGTON.OLYMPIC.120517-MAN.PDF
trying v3
14297_BAROL.PDF
trying v3
success
14297_BAROLY071108.PDF
trying 

...

success
MINPER100429.PDF
success
SILTEV090519.PDF
success
KINSAN100312.PDF
success
MILWHI06.PDF
trying v3
WADE.PACIFIC.180111-NDSMAN.PDF
trying v3
PACKARD.SPAULDING.150616-NDSMAN.PDF
success
SHEWEN090814.PDF
success
SHESHI080414.PDF
success
SHERBOURNE.SAWYER.151201-MAN.PDF
trying v3
SAWSHE05.PDF
trying v3
ORION.SHERMAN.160615-NDSMAN.PDF
success
ORION.SHERMANWAY.170322-NDSMAN.PDF
success
FLETCHER.ATWATER.PEDS.140507-MAN.PDF
success
COLFAX.HUSTON.140515-MAN.PDF
success
ALAMEDA.101FWYNBRMP.140521-MAN.PDF
success
SLAUSON.2ND.PEDS.140422-MAN.PDF
success
LACLEDE.SILVERLAKE.150617-NDSMAN.PDF
success
LACLEDE.MINNEAPOLIS.150617-NDSMAN.PDF
success
LACMIN100429.PDF
success
LACMIN081218.PDF
success
ALAMEDA.ELMONTEBUSWAY.140528-MAN.PDF
success
MELOXF05.PDF
Current Count:8501
trying v3
LACMEL090331.PDF
success
LINMEC090928.PDF
success
MEASES081223.PDF
success
MEAPIC080724.PDF
success
MCNSAT090520.PDF
success
SHA7ST06.PDF
trying v3
SEPWYA081103.PDF
success
SEPULVEDA.WYANDOTTE.170322-NDSMAN.PDF
succes

trying v3
DEARES090212.PDF
success
DEARES080114.PDF
success
DEAWIL081124.PDF
success
DEGVAN090219.PDF
success
DELHAZ080410.PDF
success
DELHYP06.PDF
trying v3
HYPERION.DELONGPRE.170307-NDSMAN.PDF
success
DELLOB05.PDF
trying v3
DELMYR100706.PDF
success
DELSAN081223.PDF
success
DELSIE081103.PDF
success
DELTOP091202.PDF
success
DEN18207.PDF
trying v3
DENKER.182ND.171031-NDSMAN.PDF
trying v3
DENTUL07.PDF
trying v3
DESGAU06.PDF
trying v3
DEVOSO090911.PDF
success
OSO.DEVONSHIRE.171129-NDSMAN.PDF
success
7298_DEWOLY161004.PDF
trying v3
DEWOLY090203.PDF
success
DILKRA07.PDF
trying v3
DODWES080206.PDF
success
DONECH090505.PDF
success
DONVEN100125.PDF
success
DONVEN07.PDF
trying v3
DOUMON090408.PDF
success
DSTFIG100106.PDF
success
EASWAS090204.PDF
success
EDEING091015.PDF
success
EDGWIN100630.PDF
success
PASSTR090923.PDF
success
PARWEN091223.PDF
success
EFFLEM090505.PDF
success
ELCYUC090428.PDF
success
ELDORO091222.PDF
success
ELLPIC080801.PDF
success
ELMSAT081201.PDF
success
ELMER.SATICOY.170315

success
CORBIN.KITTRIDGE.150429-NDSMAN.PDF
success
ROSCOE.SEDAN.150430-NDSMAN.PDF
success
CHATSWORTH.AQUEDUCT.150429-NDSMAN.PDF
success
DEERING.SATICOY.150428-NDSMAN.PDF
success
RESEDA.WINFORD.150429-NDSMAN.PDF
success
ZELZAH.KINGSBURY.150429-NDSMAN.PDF
success
SATICOY.GLADE.150428-NDSMAN.PDF
success
VERNON.HARVARD.150611-NDSMAN.PDF
success
PLATT.SHERMAN.160608-NDSMAN.PDF
success
PLATT.SHERMANWAY.150430-NDSMAN.PDF
trying v3
TOPANGACYN.HIAWATHA.150429.NDSMAN.PDF
success
TRINITY.ADAMS.150611-NDSMAN.PDF
success
SHERMANWAY.BAIRD.150820-MAN.PDF
success
12TH.ARLINGTON.151201-NDSMAN.PDF
success
ARLINGTON.12TH.150819-MAN.PDF
success
102ND.BROADWAY.150611-NDSMAN.PDF
success
PICO.STANDREWS.150616-NDSMAN.PDF
success
6TH.CURSON.150616-NDSMAN.PDF
success
OTSEGO.LANKERSHIM.150616-NDSMAN.PDF
success
SANVICENTE.TREMAINE.150616-NDSMAN.PDF
success
AVENUE52.GLENELLEN.150616-NDSMAN.PDF
success
LASALLE.25TH.150611-NDSMAN.PDF
success
69TH.NORMANDIE.150611-NDSMAN.PDF
success
SHERMANWAY.COZYCROFT.150616-NDSMA

trying v3
ABBOTKINNEY.RIALTO.PALMS.160607-NDSMAN.PDF
success
ARDMORE.OAKWOOD.160607-NDSMAN.PDF
success
BAYPARK.PLAYAVISTA.160608-NDSMAN.PDF
success
BELLEGELADE.ALHAMBRA.160614-NDSMAN.PDF
success
BEVERLY.KENMORE.160614-NDSMAN.PDF
success
CENTINELA.PENNSYLVANIA.IOWA.160607-NDSMAN.PDF
success
ENCINO.SHERMAN.160607-NDSMAN.PDF
success
FAIR.OXNARD.160608-NDSMAN.PDF
success
HIGHLAND.LELAND.160615-NDSMAN.PDF
trying v3
HIGHLAND.WARING.160614-NDSMAN.PDF
success
HIGHLAND.WARING.170307-NDSMAN.PDF
success
HOOVER.84TH_SOUTH LEG.160614-NDSMAN.PDF
Current Count:9201
success
LAURELCANYON.DONAEMILIA.160615-NDSMAN.PDF
success
LEIMERT.43RD.160618-NDSMAN (1ST DAY).PDF
trying v3
LEIMERT.43RD.160619-NDSMAN (2ND DAY).PDF
trying v3
LONGBOW.SCADLOCK.160607-NDSMAN.PDF
success
MASON.CELTIC.160615-NDSMAN.PDF
success
NATIONAL.CLARINGTON.160607-NDSMAN.PDF
success
NORMANDIE.57TH_SOUTH LEG.160614-NDSMAN.PDF
success
PONCE.VENTURA.160607-NDSMAN.PDF
success
SPAULDING.THURMAN.VENICE.160614-NDSMAN.PDF
success
102ND_DWY.COM

trying v3
PERSHING.MANCHESTER.170718.MANNDS.PDF
trying v3
WASHINGTON.EO 23RD.170805.MIOMSN.PDF
trying v3
WASHINGTON.EO 23RD.170806.MIOMAN.PDF
trying v3
WOODLEY.LEADWELL.170301-MANMIO.PDF
success
FAIRFAX.SMILEY.171004-MAN.PDF
success
GLENDALE.BRUNSWICK.171025-NDSMAN.PDF
success
MIRALESTE.CHANDELEUR.171026-NDSMAN.PDF
success
PACIFIC.28TH.171026-NDSMAN.PDF
success
GORMAN.103RD.171025-NDSMAN.PDF
trying v3
8319_6THMAP150430.PDF
success
8697_7THMAP150430.PDF
success
8734_7THWAL150430.PDF
success
8916_6THWAL150430.PDF
success
5204_DEGROD161004.PDF
trying v3
ALTAVISTA.BEVERLY.171031-NDSMAN.PDF
success
WILMINGTON.103RD.171025-NDSMAN.PDF
trying v3
VICTORIA_WEST JOG.FLORENCE AVE.171026-NDSMAN.PDF
success
VICTORIA_EAST JOG.FLORENCE.171026-NDSMAN.PDF
success
VENICE.MILITARY.171128-NDSMAN.PDF
Current Count:9401
success
CATALINA.37TH.171026-NDSMAN.PDF
trying v3
ISLAND.HARRYBRIDGES.171026-NDSMAN.PDF
success
NORMANDIE.255TH.171026-NDSMAN.PDF
success
HAYVENHURST.ESCALON.171025-NDSMAN.PDF
success
HAYVENH